In [71]:
import re
import os
import json
import time
from PIL import Image

import torch
import numpy as np
from tqdm import tqdm
from scipy.stats import ttest_ind

import faiss
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer

root = "results/"
image_root = "F:/train_blocks"

In [72]:
tokenizer = AutoTokenizer.from_pretrained("flax-community/clip-rsicd-v2")

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [74]:
model = CLIPModel.from_pretrained("flax-community/clip-rsicd-v2")

In [75]:
processor = CLIPProcessor.from_pretrained("flax-community/clip-rsicd-v2")

In [76]:
def cap_post_process(caption):
    answers = []
    
    caps = re.split("CAP\s*\d*\s*[:=-]", caption)[1:]
    for cap in caps:
        if re.search("\d{2,}", cap):
            continue
        cap = cap.strip()
        cap = re.sub("^[\'\"]", "", cap)
        cap = re.sub("[\'\"]$", "", cap)
        answers.append(cap)
    return answers

def record_post_process(record):
    answers = {}
    
    for key, item in record.items():
        extracted_captions = []
        for caption in item["response"]:
            
            extracted_captions.extend(cap_post_process(caption))
        answers[key] = extracted_captions
    
    return answers

In [77]:
records = {}

for file_name in os.listdir(root):
    file_name_capture = re.search("records_(\d_[0-9]\d?).json", file_name)
    if file_name_capture:
        file_id = file_name_capture.group(1)
        
        with open(os.path.join(root, file_name), "r") as f:
            record = record_post_process(json.load(f))
        
        if len(record) == 0:
            continue
        records[file_id] = record

In [78]:
with open("results/sample.json", "r") as f:
    samples = json.load(f)

In [79]:
images = []

for key in samples_keys:
    file_path = os.path.join(image_root, f"{key}.png")
    if os.path.isfile(file_path):
        images.append(Image.open(file_path))

In [80]:
all_captions = []
samples_indices = []
records_indices = []

for key_idx, key in tqdm(enumerate(samples_keys)):
    sim_record = {}
    for record_id, record in records.items():
        sim_record[record_id] = record.get(key, [])
        
    for record_key, captions in sim_record.items():
        record_idx = records_keys.index(record_key)
        valid_captions = [cap for cap in captions if len(tokenizer.tokenize(cap)) <= 75]
        if len(valid_captions) < 70:
            continue
            
        all_captions.extend(valid_captions)
        samples_indices.extend([key_idx] * len(valid_captions))
        records_indices.extend([record_idx] * len(valid_captions))


0it [00:00, ?it/s]
1it [00:00,  3.82it/s]
2it [00:00,  3.84it/s]
3it [00:00,  3.96it/s]
4it [00:00,  4.04it/s]
5it [00:01,  4.20it/s]
6it [00:01,  4.31it/s]
7it [00:01,  4.40it/s]
8it [00:01,  4.37it/s]
9it [00:02,  3.94it/s]
10it [00:02,  4.01it/s]
11it [00:02,  4.10it/s]
12it [00:02,  4.14it/s]
13it [00:03,  4.36it/s]
14it [00:03,  4.35it/s]
15it [00:03,  4.42it/s]
16it [00:03,  4.48it/s]
17it [00:03,  4.50it/s]
18it [00:04,  4.55it/s]
19it [00:04,  4.46it/s]
20it [00:04,  4.32it/s]
21it [00:04,  4.40it/s]
22it [00:05,  4.33it/s]
23it [00:05,  4.44it/s]
24it [00:05,  4.49it/s]
25it [00:05,  4.50it/s]
26it [00:06,  4.46it/s]
27it [00:06,  4.33it/s]
28it [00:06,  4.36it/s]
29it [00:06,  4.37it/s]
30it [00:06,  4.44it/s]
31it [00:07,  4.49it/s]
32it [00:07,  4.42it/s]
33it [00:07,  4.38it/s]
34it [00:07,  4.41it/s]
35it [00:08,  4.24it/s]
36it [00:08,  4.18it/s]
37it [00:08,  4.20it/s]
38it [00:08,  4.21it/s]
39it [00:09,  4.31it/s]
40it [00:09,  4.32it/s]
41it [00:09,  4.30it/s]
42it 

In [81]:
match_scores = np.zeros(len(all_captions))

In [82]:
start_time = time.time()
for idx, key in tqdm(enumerate(samples_keys)):
    sample_indices = [i for i, cap in enumerate(all_captions) if samples_indices[i] == idx]
    sample_captions = [all_captions[i] for i in sample_indices]
    # sample_records = [records_indices[i] for i, cap in enumerate(all_captions) if samples_indices[i] == sample_idx]

    if (match_scores[sample_indices] == 0).sum() == 0:
        print(f"idx {idx} has been done. Continue")
        continue
    
    for i in range(0, len(sample_captions), 50):
        slice_indices = sample_indices[i:i+50]
        slice_captions = sample_captions[i:i+50]
        
        inputs = processor(text=slice_captions, images=images[idx], return_tensors="pt", padding=True)
        outputs = model(**inputs).logits_per_image[0].detach().numpy()
        match_scores[slice_indices] = outputs
end_time = time.time()
print(end_time - start_time) 


0it [00:00, ?it/s]
1it [00:30, 30.35s/it]
2it [01:06, 32.11s/it]
3it [01:37, 31.81s/it]
4it [02:10, 32.22s/it]
5it [02:53, 35.22s/it]
6it [03:28, 35.28s/it]
7it [04:00, 34.31s/it]
8it [04:29, 32.76s/it]
9it [05:01, 32.37s/it]
10it [05:35, 32.93s/it]
11it [06:10, 33.46s/it]
12it [06:42, 33.10s/it]
13it [07:09, 31.45s/it]
14it [07:43, 32.22s/it]
15it [08:15, 31.95s/it]
16it [08:48, 32.21s/it]
17it [09:19, 32.02s/it]
18it [09:51, 31.82s/it]
19it [10:22, 31.82s/it]
20it [10:55, 32.03s/it]
21it [11:25, 31.43s/it]
22it [11:55, 31.14s/it]
23it [12:26, 31.00s/it]
24it [12:57, 31.08s/it]
25it [13:30, 31.62s/it]
26it [14:03, 31.84s/it]
27it [14:32, 31.26s/it]
28it [15:02, 30.87s/it]
29it [15:32, 30.53s/it]
30it [16:01, 29.96s/it]
31it [16:32, 30.46s/it]
32it [17:02, 30.26s/it]
33it [17:31, 29.88s/it]
34it [18:00, 29.63s/it]
35it [18:32, 30.28s/it]
36it [19:03, 30.49s/it]
37it [19:35, 30.87s/it]
38it [20:06, 30.86s/it]
39it [20:34, 30.27s/it]
40it [21:05, 30.34s/it]
41it [21:35, 30.23s/it]
42it 

2551.127875804901


In [83]:
for i, key in enumerate(records_keys):
    
    scores = match_scores[np.array(records_indices) == i]
    print(key)
    print(scores.shape)
    print(scores.mean())
    print(scores.std())
    

0_0
(5266,)
21.602052852313328
2.197008422019524
0_1
(8145,)
21.145144394909114
2.3749839806832513
0_10
(7643,)
21.050573482261377
2.496387817911768
0_2
(8173,)
21.116528355271612
2.341249002809695
0_3
(7584,)
21.083072877005687
2.457297100705515
0_4
(7803,)
21.12413204197515
2.473613801516891
0_5
(7908,)
20.954446864116054
2.562721101106644
0_6
(7962,)
20.973878650023273
2.5961284707319883
0_7
(6923,)
21.193824151339566
2.273691411253815
0_8
(7933,)
21.11615548493897
2.45088677412625
0_9
(7935,)
21.142657722131815
2.488920514971912


In [84]:
scores_with_indices = np.stack((match_scores, np.array(samples_indices), np.array(records_indices)))

with open('results/rsicd-clip_match_scores.npy', 'wb') as f:
    np.save(f, scores_with_indices)

In [57]:
def t_test(a, b):
    min_size = min(len(a), len(b))
    
    sample_a = a
    sample_b = b
    
    if min_size != len(sample_a):
        sample_a = np.random.choice(a, min_size, replace=False)
    if min_size != len(sample_b):
        sample_b = np.random.choice(b, min_size, replace=False)
        
    return ttest_ind(sample_a, sample_b)

In [16]:
sim_records = {}

for key in tqdm(samples.keys()):
    sim_record = {}
    for record_id, record in records.items():
        captions = record.get(key, [])
        if len(captions) == 0:
            continue
        
        inputs = processor(text=captions, images=image, return_tensors="pt", padding=True)
        logits = model(**inputs).logits_per_image[0].detach().numpy()
        sorted_logits = logits[np.argsort(logits)[::-1]]
        sim_record[record_id] = sorted_logits
    
    lengths = [value.shape[0] for value in sim_record.values()]
    
    if min(lengths) < 16:
        continue
        
    sim_records[key] = sim_record

  0%|                                                                                           | 0/85 [00:00<?, ?it/s]


NameError: name 'image' is not defined

In [21]:
sorted_logits

array([31.350168, 31.025824, 30.901922, 30.515787, 30.386406, 30.236092,
       30.185266, 30.05999 , 29.993208, 29.98081 , 29.925995, 29.623564,
       29.597374, 29.45525 , 29.43065 , 29.232107, 29.214981, 29.188152,
       29.006079, 28.98182 , 28.721973, 28.583952, 28.366207, 28.343855,
       28.219797, 28.18656 , 28.086308, 28.065895, 28.054441, 27.941397,
       27.932589, 27.834793, 27.71163 , 27.646627, 27.616869, 27.603762,
       27.496286, 27.431639, 27.413095, 27.407822, 27.279213, 27.194002,
       27.080276, 27.059748, 27.037437, 26.989027, 26.89038 , 26.798405,
       26.78367 , 26.74097 , 26.613094, 26.39619 , 26.322653, 26.301308,
       26.28823 , 26.179272, 26.138912, 26.11029 , 26.09074 , 25.8043  ,
       25.759514, 25.46328 , 25.459604, 25.423025, 25.395092, 25.23114 ,
       25.156092, 24.675657, 24.671854, 24.612137, 24.602262, 24.47859 ,
       24.261518, 24.23877 , 24.179085, 24.01067 , 23.921213, 23.321102,
       23.029516, 22.571487, 22.369669, 19.530396],

In [64]:
captions_array[sorted_indices]

array(['Several buildings form a diagonal line across the image, ranging in size from large to small.',
       'Buildings are sparsely scattered across the image, the largest one stands out in the upper right corner.',
       'The buildings in the lower half of the image form a diagonal line',
       'Aerial view reveals an intricate assortment of buildings in diamond formation, highlighting the largest structure at the top-right corner.',
       'A cluster of buildings, ranging from small to large sizes, are evenly distributed throughout the image.',
       'A dominant building stands out among smaller structures in this aerial view',
       'The organized layout of the buildings in this aerial image is accentuated by the carefully positioned small cars in the corners.',
       'The buildings in this aerial image are arranged in a diagonal line and two small cars bookend the corners.',
       'The buildings in this aerial image are arranged in a diagonal line while the small cars fram

In [12]:
kmeans = faiss.Kmeans(embedding.shape[1], 5, niter=20, verbose=True)
kmeans.train(embedding[:30])

NameError: name 'embedding' is not defined

In [79]:
index = faiss.IndexFlatL2(embedding.shape[1])
index.add(embedding)
D, I = index.search(kmeans.centroids, 2)

In [40]:
records_10.keys()

dict_keys(['10.tif_18', '10.tif_19', '10.tif_31', '10.tif_32', '10.tif_33', '10.tif_34', '10.tif_35', '10.tif_45', '10.tif_46', '10.tif_47', '10.tif_48', '10.tif_60'])

In [80]:
for i in range(I.shape[0]):
    print([captions[i] for i in I[i, :]])

['Top and bottom right buildings cluster.', 'Clustered buildings in top right and center.']
['Mostly horizontal/vertical alignment', 'Unique damage patterns top left and center']
['Damaged buildings in top left and center top.', 'Damaged buildings are concentrated in the top left and center top, indicating a possible cause or target of the damage.']
['The construction sites in the center of the image are interposed by buildings in the top right and bottom right.', 'Buildings are dispersed across the image, except for one in the middle bottom where a construction site is located.']
['Hut or tent separate in top right', 'The hut or tent in the top right stands out amidst the permanent structures, suggesting a different social or economic status.']
